In [140]:
import os

import numpy as np
from astropy.io import ascii as asc
from astropy.io import fits
from astropy.time import Time
import astropy.units as u
import astropy.constants as c
from astropy.modeling.blackbody import blackbody_lambda
from astropy.modeling import fitting, models


from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib notebook

import spectroscopy as spec
from supernova import LightCurve2

In [142]:
DATA_DIR_LCO = '../data/spectra/lco'
DATA_DIR_EFOSC = '../data/spectra/EFOSC'
DATA_DIR_FILTER = '../data/lco_filter_throughput'

FIG_DIR = '../figures'

## Spectra preliminaries

### Wavelengths are in rest frame

In [7]:
def read_iraf_spectrum(filename):
    ofile = fits.open(filename)
    flux = ofile[0].data[0,0,:]
    err = ofile[0].data[3,0,:]
    wave = spec.calc_wavelength(ofile[0].header, np.arange(len(flux))+1)
    rest_wave = spec.apply_redshift(wave, 0.0069)
    return(spec.spectrum1d(rest_wave, flux, error=err))

In [8]:
spectra_files = [
         ('asassn15oz_20150904_redblu_122216.314.fits', DATA_DIR_LCO),
         ('asassn-15oz_20150906_redblu_105042.698a.fits', DATA_DIR_LCO),
         ('asassn15oz_20150907_redblu_123835.277.fits', DATA_DIR_LCO),
         ('asassn15oz_20150911_redblu_105336.349.fits', DATA_DIR_LCO),
         ('asassn15oz_20150916_redblu_120911.274.fits', DATA_DIR_LCO),
         ('asassn-15oz_20150920_redblu_135034.512.fits',DATA_DIR_LCO),
         ('asassn-15oz_20150924_redblu_123847.580.fits',DATA_DIR_LCO),
         ('asassn-15oz_20150930_redblu_122858.217.fits',DATA_DIR_LCO),
         ('tASASSN-15oz_20151003_Gr13_Free_slit1.0_57720_1_e.fits',DATA_DIR_EFOSC),
         ('asassn15oz_20151006_redblu_101906.800.fits', DATA_DIR_LCO),
         ('asassn15oz_20151014_redblu_112918.305.fits', DATA_DIR_LCO),
         ('asassn-15oz_20151025_redblu_102221.833.fits', DATA_DIR_LCO),
         ('asassn-15oz_20151107_redblu_101210.833.fits', DATA_DIR_LCO),
         ('tASAS-SN_15oz_20151107_Gr13_Free_slit1.5_57723_1_e.fits', DATA_DIR_EFOSC),
         ('tASAS-SN_15oz_20151118_Gr13_Free_slit1.0_57723_1_e.fits', DATA_DIR_EFOSC)
                ]

## SNEC preliminaries

In [9]:
snec_dir = '../../snec_models/Ni_mass_0.0830/Ni_mixing_5.0/M18.0/E_1.400/K_10.0/R_2400/'

In [15]:
snec_temp_tb = asc.read(os.path.join(snec_dir, 'Data/T_eff.dat'), names=['time', 'Teff'])
snec_temp_tb['time'] = snec_temp_tb['time']*u.second
snec_temp_tb['Teff'] = snec_temp_tb['Teff']*u.Kelvin

In [19]:
sn15oz = LightCurve2('asassn-15oz')

In [79]:
fitter = fitting.LinearLSQFitter()
const = models.Const1D()

## Filter Preliminaries

In [118]:
filter_fnames = ['bssl-ux-022.txt',
                 'bssl-bx-004.txt',
                 'bssl-vx-022.txt',
                 'sdss.gp.txt',
                 'sdss.rp.txt',
                 'sdss.ip.txt']
filter_colors =  ['#332288','#882255', '#117733','#88CCEE','#DDCC77', '#CC6677']

#for n, i in enumerate(filter_colors):
#    plt.plot(np.arange(10)+n, 'o', color=i, label=i)
#plt.legend(loc='best')


In [157]:
with PdfPages(os.path.join(FIG_DIR, 'BB_spec_comp.pdf')) as pp:
    for ifile, idir in spectra_files:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax_throughput = ax.twinx()
        for icolor, ifilter_file in zip(filter_colors, filter_fnames):
            filter_tbdata = asc.read(os.path.join(DATA_DIR_FILTER, ifilter_file), names=['wavelength', 'throughput'], comment='\lambda')
            ax_throughput.fill_between((filter_tbdata['wavelength']*u.nm).to(u.angstrom), filter_tbdata['throughput'], 
                                       label=ifilter_file.strip('.txt'), alpha=0.5, color=icolor)
        ax_throughput.legend(bbox_to_anchor = [0.7,2.25/5, 0.3, 2/5 ])
        ax_throughput.set_ylim(ymax=5)
        spectrum = read_iraf_spectrum(os.path.join(idir, ifile))
        date_obs = Time(fits.getval(os.path.join(idir, ifile), 'date-obs', 0))
        date_obs.out_subfmt = 'date'
        phase = date_obs - Time(sn15oz.jdexpl, format='jd')
        itemp = np.interp(phase.value, (snec_temp_tb['time'].data*u.second).to(u.day), snec_temp_tb['Teff'].data)
        blackbody_flux = blackbody_lambda(spectrum.wave, itemp)
        fit_indx = spectrum.wave > 5500
        #scale_factor = fitter(const, spectrum.wave[fit_indx], blackbody_flux[fit_indx]/spectrum.flux[fit_indx])

        #bb_spec = spec.spectrum1d(spectrum.wave, blackbody_flux/scale_factor.amplitude)
        ax.plot(spectrum.wave, spectrum.flux, label='Spectrum for Day {:2.2f}'.format(phase.value))
        #ax.plot(bb_spec.wave, bb_spec.flux, label='min chisq')
        scale_spec = spec.scale_spectra(spec.spectrum1d(spectrum.wave, blackbody_flux), spectrum, wlmin=5500)
        ax.plot(scale_spec.wave, scale_spec.flux, label='Scaled Black Body')
        ax.legend(loc='upper right')
        ax_throughput.set_xlim(3000, 10000)
        ax.set_title('BB Spec Comparison for {}'.format(date_obs))
        ax.set_ylim(ymax=spectrum.flux.max()*1.1)
        pp.savefig()
        plt.close()
    
    

In [149]:
ax.legend?


In [139]:
'{}'.format(date_obs)

'2015-10-14T11:29:18.305'

In [155]:
def get_ra_dec(sql_result_dict):
    RA = []
    DEC = []
    for item in sql_result_dict:
        